# Tass


In [1]:
import requests
import numpy as np
import datetime
import re
from bs4 import BeautifulSoup

from fake_useragent import UserAgent

import time
import datetime
import pandas as pd
import pickle
import time

from tqdm import tqdm
from joblib import Parallel, delayed

from tqdm import tqdm

In [2]:
from multiprocessing.pool import ThreadPool

def Separator(vect, part):
    n = len(vect)
    vec_parts = [round(n/part)*i for i in range(part)]
    vec_parts.append(n)
    out = [vect[vec_parts[i]:vec_parts[i+1]] for i in range(part)]
    return(out)

def Map(vect, parser_function):
    out = [parser_function(item) for item in vect]
    return(out)

def Reduce(l):
    ll = [ ]
    for item in l:
        ll.extend(item)
    return(ll)

def MRDownloader(what, parts, parser_function):
    separatorlist = Separator(what, parts)
    def Mp(what):
        return Map(what, parser_function=parser_function)

    pool = ThreadPool(parts)
    l = pool.map(Mp, separatorlist)

    itog = Reduce(l)
    return(itog)    

# 1. Download hrefs

In [ ]:
def tass_lenta(before,limit=200):
    mainpage = 'http://tass.ru/api/news/lenta?limit='+str(limit)+'&before='+str(before)
    response = requests.get(mainpage)
    dic = response.json()
    cur_news = [ ]
    df = dic['articles']
    for item in df:
        try:
            new = { }
            new['title'] = item['title']
            new['category'] = item['section']['title']
            new['href'] = item['url']
            new['date'] = datetime.datetime.fromtimestamp(int(item['time'])).strftime('%Y-%m-%d %H:%M:%S')
            new['uci_time'] = int(item['time'])
            cur_news.append(new)
        except:
            print('Error! \n',item, '\n')
    return cur_news

In [ ]:
'1514764790'   # 31 декабря 2017 года 23:59:59
'1483228799'   # 2016 
'1451606399'   # 2015
'1420070399'   # 2014
'1388534399'   # 2013
'1356998399'   # 2012
'1325375999'   # 2011
'1293839999'   # 2010
'1262303999'   # 2009
'1230767999'   # 2008 

In [ ]:
# tass_news = [ ]

# before = 999999999999

for i in tqdm(range(100000)):
    current_news = tass_lenta(before)
    before = current_news[-1]['uci_time']
    
    tass_news.extend(current_news)
    if before < 1514674664:
        break 

In [ ]:
before

In [ ]:
len(tass_news)

In [ ]:
tass_news2 = [item for item in tass_news if item['date'].split('-')[0] == '2020']
len(tass_news2)

In [ ]:
tass_news3 = [item for item in tass_news2 if item['date'].split('-')[1] in ['06', '07', '08', '09', '10', '11']]
len(tass_news3)

In [ ]:
tass_news = tass_news3

In [ ]:
print('В списке сейчас: ', len(tass_news))
df = pd.DataFrame(tass_news)
print(df.shape)
print('Без дубликатов: ', len(set(df.href)))
print(df.drop_duplicates().shape)

with open('news_data/tass_titles_new.pickle', 'wb') as f:
      pickle.dump(tass_news, f)

df.tail()

``` 
   Что ещё можно достать нахаляву из df 
   {'audio': False,   'color': '2',   'flash': False,   'id': '4070830',
   'is_breaking_news': False,  'is_online': False,   'live_text': False,
   'marked': False,  'photos': False,  'search_queries': None,
   'section': {'id': '25', 'title': 'Экономика и бизнес', 'url': '/ekonomika'},
   'show_at_common_feed': True,  'show_at_section_feed': True, 'slideshow': False
   'time': '1488624568',  'title': 'Глава ВТБ: курс доллара к концу года может достигнуть 61-62 рублей',
   'topics': None,  'url': '/ekonomika/4070830', 'video': False}, 
```

# 2. Download news

In [3]:
with open('news_data/tass_titles_new.pickle', 'rb') as f:
    tass_titles = pickle.load(f)
    
len(tass_titles)

47240

In [4]:
def page_content(url):
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html,"lxml")

    vvv = soup.findAll("div", { "class" : "news" })[0]
    snippet = vvv.find_all("div", {"class": "news-header__lead"})[0].text
    text = vvv.find_all("div", {"class": "text-content"})[0].text
    return text, snippet

def get_tass_news(item_from_vect):
    url = "http://tass.ru" + item_from_vect['href']
    try:
        text, snippet = page_content(url)
        item_from_vect['text'] = text.strip()
        item_from_vect['snippet'] = snippet.strip()
        return item_from_vect
    except:
        print(url)
        return{ }

get_tass_news(tass_titles[0])

{'title': 'Полиция Украины получила восемь сообщений о минировании избирательных участков',
 'category': 'Международная панорама',
 'href': '/mezhdunarodnaya-panorama/9812053',
 'date': '2020-10-25 17:45:06',
 'uci_time': 1603637106,
 'text': 'КИЕВ, 25 октября. /ТАСС/. Национальная полиция Украины получила в воскресенье восемь сообщений о минировании избирательных участков, которые оказались ложными. Об этом сообщил на брифинге глава Нацполиции Игорь Клименко.\n"На сегодняшний день нас "минировали" уже восемь раз. Национальная полиция не останавливала процессы волеизъявления, очень быстро и оперативно проверила эти факты, они оказались фейками. Мы сейчас устанавливаем, кто звонил, с какой целью", - сказал Клименко.\nОн отметил ряд нарушений на избирательных участках. "Кандидаты на должности местных депутатов или мэров городов распространяют антисептики, ручки и другую продукцию с логотипами того или иного кандидата. Эти факты мы также жестко и быстро пресекаем", - подчеркнул Клименко.\

In [5]:
# itog_titles = [item for item in tass_titles if item['date'][:4] == "2020"]
# len(itog_titles)
itog_titles = tass_titles

In [6]:
def Separator(vect, part):
    n = len(vect)
    vec_parts = [round(n/part)*i for i in range(part)]
    vec_parts.append(n)
    out = [vect[vec_parts[i]:vec_parts[i+1]] for i in range(part)]
    return(out)

In [8]:
i = 0

x_batch = Separator(itog_titles, 500)

In [19]:
result = [ ]
for batch in tqdm(x_batch[(198+186):]):
    
    n_jobs = -1
    result_cur = Parallel(n_jobs=n_jobs)(delayed(get_tass_news)(
        text) for text in batch)

    print('Скачал батч номер ' + str(i))
    i += 1
    result.extend(result_cur)

  1%|          | 1/116 [01:16<2:26:57, 76.67s/it]

Скачал батч номер 384


  2%|▏         | 2/116 [01:41<1:56:03, 61.09s/it]

Скачал батч номер 385


  3%|▎         | 3/116 [02:39<1:53:38, 60.34s/it]

...

Скачал батч номер 497


 99%|█████████▉| 115/116 [4:50:09<02:50, 170.70s/it]

Скачал батч номер 498


100%|██████████| 116/116 [4:59:11<00:00, 154.76s/it]

Скачал батч номер 499


In [ ]:
# %%time
# itog_news = MRDownloader(itog_titles, 10, parser_function=get_tass_news)
# len(itog_news)

In [20]:
len(result)

11144

In [21]:
# хули так мало, по ходу парсер не докачивает какие-то случаи
itog_news = [itog for itog in result if len(itog.keys()) != 0 ]
len(itog_news)

6892

In [22]:
pd.DataFrame(itog_news).head()

,title,category,href,date,uci_time,text,snippet
0,"Физики узнали, почему гребни прибрежных волн с...",Наука,/nauka/8895611,2020-07-06 16:43:06,1594042986,"ТАСС, 6 июля. Британские физики выяснили, поче...","Оказалось, в этом ""виноваты"" волны и водоросли..."
1,Журналистка Прокопьева будет добиваться снижен...,Происшествия,/proisshestviya/8896443,2020-07-06 16:38:56,1594042736,"МОСКВА, 6 июля. /ТАСС/. Псковская журналистка ...",Ранее суд признал ее виновной и назначил штраф...
2,Союз индустрии гостеприимства отметил рост спр...,Экономика и бизнес,/ekonomika/8896375,2020-07-06 16:37:30,1594042650,"МОСКВА, 6 июля. /ТАСС/. Общенациональный союз ...","В частности, по данным союза, уровень брониров..."
3,"Эксперты: решение Дании по ""Северному потоку -...",Экономика и бизнес,/ekonomika/8895757,2020-07-06 16:36:00,1594042560,"МОСКВА, 6 июля. /ТАСС/. Решение Датского энерг...","При этом специалисты отмечают, что возможность..."
4,"В Москве из-за жары объявили ""оранжевый"" урове...",В стране,/v-strane/8895589,2020-07-06 16:32:49,1594042369,"МОСКВА, 6 июля. /ТАСС/. В связи с аномальной ж...",Аномально высокая температура воздуха ожидаетс...


In [23]:
with open('news_data/tass_news_new_part3.pickle', 'wb') as f:
    pickle.dump(itog_news, f)